In [1]:
#import packages
import os, csv, time
import pandas as pd
import numpy as np

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch import autocast

import torchvision.transforms as transforms

from timm.models import create_model

from sklearn.metrics import f1_score
from livelossplot import PlotLosses

from tqdm import tqdm

/users/snranepuradewage/.lico_env/jupyter/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_DIR = os.path.join("/users/snranepuradewage/roco_multimodal/baseline/models_concept/")
filenameCSV=MODEL_DIR+"EfficientNetB0_perform.csv"
DATA_DIR="/users/snranepuradewage/roco-dataset-master/data-master"
data_train=DATA_DIR+"/train/"
data_valid=DATA_DIR+"/valid/"

In [3]:
#load dataset with IDs
df_train=pd.read_csv(DATA_DIR+"/train_concepts.csv",sep=",")
df_valid=pd.read_csv(DATA_DIR+"/valid_concepts.csv",sep=",")

In [4]:
df_train["image_path"]=data_train+df_train.ID+".jpg"
df_valid["image_path"]=data_valid+df_valid.ID+".jpg"

In [5]:
df_train_merged=pd.concat([df_train,df_valid])

In [6]:
cuis_list=[]

In [7]:
for (i,row) in df_train_merged.iterrows():
    for cui in row["CUIs"].split(";"):
        if not cui in cuis_list:
            cuis_list.append(cui)

In [8]:
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [9]:
NUM_CLASSES=len(cuis_list)

In [10]:
print("Total unique CUIs:", len(cuis_list))

Total unique CUIs: 1947


In [10]:
class ROCOv2Dataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform # Image augmentation pipeline

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        obj = self.data.iloc[index] # get instance
        label = obj.CUIs # get label
        label_enc=torch.zeros((NUM_CLASSES))
        for cui in label.split(";"):
            label_enc[cuis_list.index(cui)]=1
        # img. augmentation
        img = Image.open(obj.image_path).convert("RGB") # load image
        img = self.transform(img)

        return (img, label_enc)


In [11]:
class IdentityTransform:
    def __call__(self, x):
        return x


# train data augmentation/ preprocessing pipeline
def get_train_augmentation_preprocessing(img_size, rand_aug=False):
    print(f'IMG_SIZE_TRAIN: {img_size}, RandAug: {rand_aug}')
    return transforms.Compose([
                transforms.Resize(int(img_size * 1.25)), # Expand IMAGE_SIZE before random crop
                #RandomGridShuffle(grid=TRANSFORMS['n_grid']),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomCrop((img_size, img_size)), # Random Crop to IMAGE_SIZE
                #transforms.RandAugment(num_ops=2, magnitude=9) if rand_aug else IdentityTransform(),
                transforms.ToTensor(),
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
            ])

In [12]:
# shuffle
df_train_merged = df_train_merged.sample(frac=1, random_state=1).reset_index(drop=True)


In [13]:
imgsize_train=224
imgsize_val=224

In [14]:
train_aug_preprocessing = get_train_augmentation_preprocessing(imgsize_train, True)

IMG_SIZE_TRAIN: 224, RandAug: True


In [15]:
train_dataset= ROCOv2Dataset(df_train_merged, transform = train_aug_preprocessing)


In [16]:
BATCH_SIZE=256

In [17]:
train_loader = DataLoader(dataset = train_dataset, shuffle = True, batch_size = BATCH_SIZE, num_workers = 9, drop_last = True, pin_memory = True)

In [ ]:
m = nn.Sigmoid()
lr=0.001
opt="adam"
use_amp = True
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
batchsize_factor=1
val_interval = 1
epoch_loss_values = []
max_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=create_model('efficientnet_b0', pretrained=True, num_classes=NUM_CLASSES, drop_path_rate=0.2)
loss_function = torch.nn.MultiLabelSoftMarginLoss()
if opt =="adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
elif opt=="sgd":
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
else:
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
model = model.to(device)
for epoch in range(max_epochs):
    start_time = time.time()
    model.train()
    logs = {}
    epoch_loss = 0
    epoch_loss_val = 0
    step = 0
    labels_sum = np.empty([0, NUM_CLASSES])
    pred_sum = np.empty([0, NUM_CLASSES])
    labels_sum_val = np.empty([0, NUM_CLASSES])
    pred_sum_val = np.empty([0, NUM_CLASSES])
    
    for batch_idx, (inputs, labels) in enumerate(
        tqdm(train_loader, desc=f"Epoch [{epoch+1}/{max_epochs}]", leave=False)):
        step += 1
        inputs = inputs.cuda()
        labels = labels.cuda()
        with autocast(device_type = 'cuda', enabled = True): 
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss = loss / batchsize_factor
            scaler.scale(loss).backward()
            output_sig = m(outputs)
            output_sig_class = (output_sig >= 0.5).long()
        if (step + 1) % batchsize_factor == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
        epoch_loss += (loss.item() * batchsize_factor)
        epoch_len = len(train_dataset) // train_loader.batch_size
        labels_sum = np.append(labels_sum, labels.detach().cpu().numpy(), axis = 0)

        pred_sum = np.append(pred_sum, output_sig_class.detach().cpu().numpy(), axis = 0)

    logs['log loss'] = epoch_loss / len(train_dataset)
    logs['F1 macro'] = f1_score(labels_sum.T, pred_sum.T, average = 'macro')
    epoch_loss_values.append(epoch_loss)
    
    torch.save(model.state_dict(), MODEL_DIR + "model_" + str(opt) + "_" + str(lr) + "_" + str(epoch) + ".pth")
    
    elapsed = time.time() - start_time
    print(f"Epoch {epoch+1}/{max_epochs} | Loss: {logs['log loss']:.4f} | F1: {logs['F1 macro']:.4f} | Time: {elapsed:.1f}s")
    
    with open(filenameCSV, "a", newline="") as f:
        csv.writer(f).writerow([epoch + 1, logs['log loss'], logs['F1 macro'], elapsed])

/tmp/ipykernel_41793/4069333915.py:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


Epoch 1/20 | Loss: 0.0001 | F1: 0.2911 | Time: 102.5s


Epoch 2/20 | Loss: 0.0000 | F1: 0.5383 | Time: 103.0s


Epoch 3/20 | Loss: 0.0000 | F1: 0.5614 | Time: 129.2s


Epoch 4/20 | Loss: 0.0000 | F1: 0.5740 | Time: 120.2s


Epoch 5/20 | Loss: 0.0000 | F1: 0.5804 | Time: 124.2s


Epoch [6/20]:   0%|          | 1/272 [00:01<08:32,  1.89s/it]